In [1]:
!pip install SoccerNet --upgrade

  Attempting uninstall: SoccerNet
    Found existing installation: SoccerNet 0.1.60
    Uninstalling SoccerNet-0.1.60:
      Successfully uninstalled SoccerNet-0.1.60


In [3]:
import SoccerNet
from SoccerNet.Downloader import SoccerNetDownloader
mySoccerNetDownloader=SoccerNetDownloader(LocalDirectory="/FAIData/videos")

In [5]:
mySoccerNetDownloader.password = "s0cc3rn3t"
mySoccerNetDownloader.downloadGames(files=["1_720p.mkv", "2_720p.mkv"], split=["train","valid","test","challenge"])

KeyboardInterrupt: 

In [ ]:
mySoccerNetDownloader.downloadDataTask(task="spotting-ball-2023", split=["train", "valid", "test", "challenge"], password="s0cc3rn3t")


  0%|          | 0.00/3.94G [00:00<?, ?iB/s]



  0%|          | 0.00/1.92G [00:00<?, ?iB/s]


In [8]:
mySoccerNetDownloader.downloadGames(files=["Labels-v2.json"], split=["train","valid","test"])


  0%|          | 0.00/37.6k [00:00<?, ?iB/s]

  0%|          | 0.00/46.8k [00:00<?, ?iB/s]

  0%|          | 0.00/32.7k [00:00<?, ?iB/s]

  0%|          | 0.00/38.8k [00:00<?, ?iB/s]

  0%|          | 0.00/49.7k [00:00<?, ?iB/s]

  0%|          | 0.00/46.1k [00:00<?, ?iB/s]

  0%|          | 0.00/35.3k [00:00<?, ?iB/s]

  0%|          | 0.00/45.5k [00:00<?, ?iB/s]

  0%|          | 0.00/51.3k [00:00<?, ?iB/s]

  0%|          | 0.00/43.4k [00:00<?, ?iB/s]

  0%|          | 0.00/42.6k [00:00<?, ?iB/s]

  0%|          | 0.00/52.6k [00:00<?, ?iB/s]

  0%|          | 0.00/50.0k [00:00<?, ?iB/s]

  0%|          | 0.00/35.9k [00:00<?, ?iB/s]

  0%|          | 0.00/48.4k [00:00<?, ?iB/s]

  0%|          | 0.00/51.3k [00:00<?, ?iB/s]

  0%|          | 0.00/44.6k [00:00<?, ?iB/s]

  0%|          | 0.00/42.8k [00:00<?, ?iB/s]

  0%|          | 0.00/32.9k [00:00<?, ?iB/s]

  0%|          | 0.00/40.7k [00:00<?, ?iB/s]

  0%|          | 0.00/35.7k [00:00<?, ?iB/s]

  0%|          | 0.00/47.1k [00:0

In [12]:
import os
import json
import cv2

def extract_goal_events_with_context(data, context_time=10000):
    goal_events = []


    for annotation in data["annotations"]:
        if annotation["label"] == "Goal":
            goal_position = int(annotation["position"])
            start_time = max(0, goal_position - context_time)
            end_time = goal_position + context_time

            goal_events.append({
                "label": "Goal",
                "goal_position": goal_position,
                "start_time": start_time,
                "end_time": end_time
            })

    return goal_events


def extract_frames_in_timeframe(video_path, start_time, end_time, output_dir, frame_rate=30):
    # Open the video file
    video = cv2.VideoCapture(video_path)

    if not video.isOpened():
        print(f"Error: Could not open video {video_path}")
        return []

    # Convert milliseconds to seconds for OpenCV
    start_time_sec = start_time / 1000.0
    end_time_sec = end_time / 1000.0

    # Set the video position to the start time
    video.set(cv2.CAP_PROP_POS_MSEC, start_time_sec * 1000)

    frames = []
    frame_count = 0
    success = True

    # Calculate the number of frames to extract
    total_frames_to_extract = int((end_time_sec - start_time_sec) * frame_rate)

    while success and frame_count < total_frames_to_extract:
        success, frame = video.read()

        if not success:
            break

        # Save each frame as an image and keep track of it
        frame_file = os.path.join(output_dir, f"frame_{frame_count}.jpg")
        cv2.imwrite(frame_file, frame)
        frames.append(frame_file)  # Save file path for later use

        frame_count += 1

    video.release()
    print(f"Extracted {frame_count} frames from {start_time_sec} to {end_time_sec} seconds.")
    return frames


def create_video_from_frames(frame_files, output_video_path, frame_rate=30):
    if not frame_files:
        print(f"No frames to create video at {output_video_path}.")
        return

    # Read the first frame to get the frame size
    first_frame = cv2.imread(frame_files[0])
    height, width, layers = first_frame.shape

    # Define the video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 'mp4v' for .mp4 format
    video_writer = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (width, height))

    for frame_file in frame_files:
        frame = cv2.imread(frame_file)
        video_writer.write(frame)

    video_writer.release()
    print(f"Video created at {output_video_path}")

def extract_goal_frames(data, video_path, output_dir, context_time=10000):
    goal_events = []

    # Loop through annotations to find 'Goal' events
    for annotation in data["annotations"]:
        if annotation["label"] == "Goal":
            # Extract current position of the 'Goal' event
            goal_position = int(annotation["position"])

            # Calculate start and end time (10 seconds before and after)
            start_time = max(0, goal_position - context_time)
            end_time = goal_position + context_time

            goal_events.append({
                "label": "Goal",
                "goal_position": goal_position,
                "start_time": start_time,
                "end_time": end_time
            })

            # Create directory for goal frames
            goal_output_dir = os.path.join(output_dir, f"goal_at_{goal_position}")
            if not os.path.exists(goal_output_dir):
                os.makedirs(goal_output_dir)

            # Extract frames for this goal event
            frames = extract_frames_in_timeframe(video_path, start_time, end_time, goal_output_dir)

            # Collate the frames into a video
            if frames:
                output_video_path = os.path.join(output_dir, f"goal_at_{goal_position}.mp4")
                create_video_from_frames(frames, output_video_path)


    return goal_events

with open("/FAIData/videos/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/Labels-v2.json") as f:
  data = json.load(f)
  print(data)


video_path = "/FAIData/videos/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/1_720p.mkv"

output_dir = "/FAIData/videos/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/extracted"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)


goal_events = extract_goal_frames(data, video_path, output_dir)

if not goal_events:
    print("No 'Goal' events found.")
else:
    print(f"Extracted frames for {len(goal_events)} 'Goal' events.")

{'UrlLocal': 'england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/', 'UrlYoutube': '', 'annotations': [{'gameTime': '1 - 00:00', 'label': 'Kick-off', 'position': '0', 'team': 'away', 'visibility': 'visible'}, {'gameTime': '1 - 02:13', 'label': 'Ball out of play', 'position': '133295', 'team': 'not applicable', 'visibility': 'visible'}, {'gameTime': '1 - 02:29', 'label': 'Throw-in', 'position': '149168', 'team': 'away', 'visibility': 'visible'}, {'gameTime': '1 - 02:36', 'label': 'Ball out of play', 'position': '156098', 'team': 'not applicable', 'visibility': 'visible'}, {'gameTime': '1 - 03:02', 'label': 'Corner', 'position': '182775', 'team': 'away', 'visibility': 'visible'}, {'gameTime': '1 - 04:06', 'label': 'Ball out of play', 'position': '246619', 'team': 'not applicable', 'visibility': 'visible'}, {'gameTime': '1 - 04:18', 'label': 'Throw-in', 'position': '258040', 'team': 'home', 'visibility': 'not shown'}, {'gameTime': '1 - 04:30', 'label': 'Shots on target', 'posit

# Extract for not just GOAL but all Key Moments (Added by Aatmaj)

Extracted 300 frames from 0.0 to 10.0 seconds.
Video created at /FAIData/videos/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/extracted\Kick-off_at_0.mp4
Extracted 599 frames from 123.295 to 143.295 seconds.
Video created at /FAIData/videos/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/extracted\Ball out of play_at_133295.mp4
Extracted 600 frames from 139.168 to 159.168 seconds.
Video created at /FAIData/videos/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/extracted\Throw-in_at_149168.mp4
Extracted 600 frames from 146.098 to 166.098 seconds.
Video created at /FAIData/videos/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/extracted\Ball out of play_at_156098.mp4
Extracted 600 frames from 172.775 to 192.775 seconds.
Video created at /FAIData/videos/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/extracted\Corner_at_182775.mp4
Extracted 600 frames from 236.619 to 256.619 seconds.
Video created at /FAIData/vid

KeyboardInterrupt: 

In [5]:
import os
import json
import cv2


# Rankings of event labels
RANKINGS = {
    "Kick-off": 1,
    "Goal": 2,
    "Shots on target": 3,
    "Red card": 4,
    "Corner": 5,
    "Yellow card": 6,
    "Shots off target": 7,
    "Foul": 8,
    "Direct free-kick": 9,
    "Offside": 10,
    "Clearance": 11,
    "Indirect free-kick": 12,
    "Throw-in": 13,
    "Ball out of play": 14,
    "Substitution": 15
}

def extract_events_with_context(data, context_time=10000):
    events = []

    for idx, annotation in enumerate(data["annotations"]):
        event_label = annotation["label"]
        event_position = int(annotation["position"])
        start_time = max(0, event_position - context_time)
        end_time = event_position + context_time


        events.append({
            "label": event_label,
            "event_position": event_position,
            "start_time": start_time,
            "end_time": end_time,
            "gameTime": annotation["gameTime"],
            "index": idx
        })

    return events


def extract_frames_in_timeframe(video_path, start_time, end_time, frame_rate=30):
    video = cv2.VideoCapture(video_path)

    if not video.isOpened():
        print(f"Error: Could not open video {video_path}")
        return []

    start_time_sec = start_time / 1000.0
    end_time_sec = end_time / 1000.0

    video.set(cv2.CAP_PROP_POS_MSEC, start_time_sec * 1000)

    frames = []
    frame_count = 0
    success = True

    total_frames_to_extract = int((end_time_sec - start_time_sec) * frame_rate)

    while success and frame_count < total_frames_to_extract:
        success, frame = video.read()

        if not success:
            break

        frames.append(frame)

        frame_count += 1

    video.release()
    print(f"Extracted {frame_count} frames from {start_time_sec} to {end_time_sec} seconds.")
    return frames


def create_video_from_frames(frames, output_video_path, frame_rate=30):
    if not frames:
        print(f"No frames to create video at {output_video_path}.")
        return

    height, width, layers = frames[0].shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (width, height))

    for frame in frames:
        video_writer.write(frame)

    video_writer.release()
    print(f"Video created at {output_video_path}")


def extract_event_frames(data, video_path, output_dir, context_time=10000):
    events = extract_events_with_context(data, context_time)
    print(video_path)

    for event in events:
        event_label = event["label"]
        event_position = event["event_position"]
        start_time = event["start_time"]
        end_time = event["end_time"]
        half = event["gameTime"][:1]
        unique_index = event["index"]
        
        path = os.path.join(video_path, f"{half}_720p.mkv")
        if not os.path.exists(path):
            print(f"Error: Video file {path} does not exist.")
            continue

        label_rank = RANKINGS.get(event_label, 15)  # Default rank to 15 if label is not in RANKINGS
        folder_name = event_label if label_rank <= 10 else "nothing"

        # Create label-specific folder if not exists
        label_dir = os.path.join(output_dir, folder_name)
        if not os.path.exists(label_dir):
            os.makedirs(label_dir)

        # Extract frames
        frames = extract_frames_in_timeframe(path, start_time, end_time)

        # Create a video for the event
        if frames:
            output_video_path = os.path.join(label_dir, f"{folder_name}_half_{half}_at_{event_position}_index_{unique_index}.mp4")
            create_video_from_frames(frames, output_video_path)

    return events


# # Load the JSON data
# with open("/FAIData/videos/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/Labels-v2.json") as f:
#     data = json.load(f)

# # Define video path and output directory
# video_path = "/FAIData/videos/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/1_720p.mkv"
# output_dir = "/FAIData/videos/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/extracted"

# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# # Extract frames for all event types and store videos in respective label folders
# event_frames = extract_event_frames(data, video_path, output_dir)

# if not event_frames:
#     print("No events found.")
# else:
#     print(f"Extracted videos for {len(event_frames)} events.")


def process_league_videos(root_path):
    for league in os.listdir(root_path):
        league_path = os.path.join(root_path, league)
        if not os.path.isdir(league_path):
            continue

        for season in os.listdir(league_path):
            season_path = os.path.join(league_path, season)
            if not os.path.isdir(season_path):
                continue

            for match in os.listdir(season_path):
                match_path = os.path.join(season_path, match)
                if not os.path.isdir(match_path):
                    continue

                # Load the JSON file
                label_file_path = os.path.join(match_path, "Labels-v2.json")
                if not os.path.exists(label_file_path):
                    print(f"No label file found in {match_path}. Skipping...")
                    continue

                with open(label_file_path) as f:
                    data = json.load(f)

                # Define output directory
                output_dir = os.path.join(root_path, "extracted")
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)

                # Extract frames and create videos
                print(f"Processing match: {match}")
                extract_event_frames(data, match_path, output_dir)


# Root path for the videos
root_path = "/FAIData/videos/"

# Process all league videos
process_league_videos(root_path)


Processing match: 2015-02-21 - 18-00 Chelsea 1 - 1 Burnley
/FAIData/videos/england_epl\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley
Extracted 300 frames from 0.0 to 10.0 seconds.
Video created at /FAIData/videos/extracted\Kick-off\Kick-off_half_1_at_0_index_0.mp4
Extracted 599 frames from 123.295 to 143.295 seconds.
Video created at /FAIData/videos/extracted\nothing\nothing_half_1_at_133295_index_1.mp4
Extracted 600 frames from 139.168 to 159.168 seconds.
Video created at /FAIData/videos/extracted\nothing\nothing_half_1_at_149168_index_2.mp4
Extracted 600 frames from 146.098 to 166.098 seconds.
Video created at /FAIData/videos/extracted\nothing\nothing_half_1_at_156098_index_3.mp4
Extracted 600 frames from 172.775 to 192.775 seconds.
Video created at /FAIData/videos/extracted\Corner\Corner_half_1_at_182775_index_4.mp4
Extracted 600 frames from 236.619 to 256.619 seconds.
Video created at /FAIData/videos/extracted\nothing\nothing_half_1_at_246619_index_5.mp4
Extracted 600 frames f

KeyboardInterrupt: 